In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval

# Análise Exploratória de Dados Imobiliários

## 01. Conjunto de Dados : Contexto


Escrever uma breve descrição sobre os dados e algumas hipóteses

In [2]:
pd.set_option('display.max_columns', 30)

In [3]:
df = pd.read_csv('dados_imoveis_sp.csv')
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,"['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...",['101'],2574084550,[1],"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",[],[2],['111'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['140'],2583748663,[2],"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",[2],[4],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['50'],2562971980,[1],"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",[0],[1],['50'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,"['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...",['58'],2580478200,[1],"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",[],[1],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,"['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...",['64'],2583729583,[1],"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",[],[1],['80'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


## 02. Limpeza e Tratamento de dados

Os tipos de dados em todas as colunas deste dataset estão no formato **string**. Os dados numéricos precisam de uma **conversão de tipo**.

In [4]:
cols = ['usableAreas','parkingSpaces','suites','bathrooms','totalAreas','bedrooms']

for var in cols:
    s_extracted_digits = df[var].str.extract('\[[\']{0,1}(\d*)[\']{0,1}\]').squeeze()
    df[var] = s_extracted_digits.apply(lambda x: int(x) if x.isdigit() else np.nan)
    
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,"['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...",101,2574084550,1.0,"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",NaN,2,111.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",140,2583748663,2.0,"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",2.0,4,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",50,2562971980,1.0,"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",0.0,1,50.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,"['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...",58,2580478200,1.0,"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",NaN,1,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,"['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...",64,2583729583,1.0,"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",NaN,1,80.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


Diversas colunas também possuem bastante informações que nos ajudaram a melhor caracterizar cada imóvel.
No entanto, essas informações estão em formato bruto, "raw data", e precisam de um tratamento.

A coluna *'address'* possui características que lembram um dicionário. Mas seu tipo é string. 
Precisamos tratar essas linhas para navegar e extrair seus atributos com facilidade.

In [5]:
print(df['address'][0],'\n')
print('Tipo de dado:',type(df['address'][0]))

{'country': 'BR', 'zipCode': '04734003', 'geoJson': '', 'city': 'São Paulo', 'streetNumber': '1850', 'level': 'STREET', 'precision': 'ROOFTOP', 'confidence': 'VALID_STREET', 'stateAcronym': 'SP', 'source': 'CORREIOS', 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282}, 'ibgeCityId': '', 'zone': 'Zona Sul', 'street': 'Avenida Adolfo Pinheiro', 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro', 'district': '', 'name': '', 'state': 'São Paulo', 'neighborhood': 'Santo Amaro', 'poisList': ['BS:Graham Bell C/B', 'BS:Graham Bell B/C', 'BS:Rua Verbo Divino, 61', 'BS:Américo Brasiliense C/B', 'BS:Parada Marechal Deodoro 2 - B/C', 'TS:Graham Bell C/B', 'TS:Graham Bell B/C', 'TS:Rua Verbo Divino, 61', 'TS:Américo Brasiliense C/B', 'TS:Parada Marechal Deodoro 2 - B/C', 'CS:7 Molinos', 'CS:Casa de Pães Neblina Paulista', 'CS:Casa de Bolo', 'CS:Berna', 'CS:Gêmel', 'VP:Kennel Club'], 'pois': [], 'valuableZones': [{'city': 'São Paulo', 'zone': 'Zona Sul', 'name': 'Cháca

O comando **literal_eval** é um interessante comando da biblioteca **ast – Abstract Syntax Trees**. 

Ele avalia uma string contendo uma expressão Python e a executa.

In [6]:
print(literal_eval(df['address'][0]),'\n')
print('Tipo de dado:',type(literal_eval(df['address'][0])))

{'country': 'BR', 'zipCode': '04734003', 'geoJson': '', 'city': 'São Paulo', 'streetNumber': '1850', 'level': 'STREET', 'precision': 'ROOFTOP', 'confidence': 'VALID_STREET', 'stateAcronym': 'SP', 'source': 'CORREIOS', 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282}, 'ibgeCityId': '', 'zone': 'Zona Sul', 'street': 'Avenida Adolfo Pinheiro', 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro', 'district': '', 'name': '', 'state': 'São Paulo', 'neighborhood': 'Santo Amaro', 'poisList': ['BS:Graham Bell C/B', 'BS:Graham Bell B/C', 'BS:Rua Verbo Divino, 61', 'BS:Américo Brasiliense C/B', 'BS:Parada Marechal Deodoro 2 - B/C', 'TS:Graham Bell C/B', 'TS:Graham Bell B/C', 'TS:Rua Verbo Divino, 61', 'TS:Américo Brasiliense C/B', 'TS:Parada Marechal Deodoro 2 - B/C', 'CS:7 Molinos', 'CS:Casa de Pães Neblina Paulista', 'CS:Casa de Bolo', 'CS:Berna', 'CS:Gêmel', 'VP:Kennel Club'], 'pois': [], 'valuableZones': [{'city': 'São Paulo', 'zone': 'Zona Sul', 'name': 'Cháca

A coluna *'pricingInfos'* pode possuir dois dicionários. Um contendo preço de aluguel, e outro com preço de compra.
Estamos interessados em pegar apenas o preço de aluguel.

In [7]:
literal_eval(df['pricingInfos'][91])

[{'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR']},
  'yearlyIptu': '120',
  'price': '410000',
  'businessType': 'SALE',
  'monthlyCondoFee': '700'},
 {'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR'],
   'monthlyRentalTotalPrice': '2400'},
  'yearlyIptu': '120',
  'price': '1700',
  'businessType': 'RENTAL',
  'monthlyCondoFee': '700'}]

A coluna *'amenities'* apresenta as facilidades que cada imóvel pode oferecer.
- Como podemos contar a frequência para nosso conjunto dados?

In [8]:
df['amenities'][:5]

0    ['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...
1    ['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...
2    ['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...
3    ['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...
4    ['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...
Name: amenities, dtype: object

In [9]:
def extract_neighborhood(address):
    address = literal_eval(address)
    neighborhood = address['neighborhood']
    
    return neighborhood

In [10]:
def extract_zone(address):
    address = literal_eval(address)
    zone = address['zone']
    
    return zone

In [11]:
def extract_zipcode(address):
    address = literal_eval(address)
    zipCode = address['zipCode']
    
    return zipCode

In [12]:
def get_rental_price(pricingInfos):
    price = [info['price'] for info\
     in literal_eval(pricingInfos)\
     if info['businessType'] == 'RENTAL'][0]
    
    return float(price)

In [13]:
def strings_para_lista(string):
    lista = string.replace(' ','').replace("'","")
    lista = lista.replace('[','').replace(']','').split(',')
    
    return lista

In [14]:
df['amenities'] = df['amenities'].apply(strings_para_lista)

In [15]:
print('Top 10 Facilidades dos Imóveis de São Paulo:')
print(df['amenities'].explode().value_counts()[:10])

Top 10 Facilidades dos Imóveis de São Paulo:
ELEVATOR           5129
POOL               4613
PARTY_HALL         4110
BARBECUE_GRILL     3961
SERVICE_AREA       3729
GYM                3687
PLAYGROUND         3357
GARDEN             3203
INTERCOM           2899
GATED_COMMUNITY    2854
Name: amenities, dtype: int64


In [16]:
top10_amenities = list(df['amenities'].explode().value_counts()[:10].index)

In [17]:
def has_amenity(amenities,amenity):
    if amenity in amenities:
        return 1
    else:
        return 0

In [18]:
# Cria uma coluna para cada uma das 10 principais facilidades
# com valor binário, representando ausência ou ocorrência da mesma
for amenity in top10_amenities:
    df[amenity.lower()] = df['amenities'].apply(has_amenity,amenity=amenity)

In [19]:
df['zipCode'] = df['address'].apply(extract_zipcode)
df['zone'] = df['address'].apply(extract_zone)
df['neighborhood'] = df['address'].apply(extract_neighborhood)

In [20]:
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood
0,"[PETS_ALLOWED, ELEVATOR, GARDEN, ELECTRONIC_GA...",101,2574084550,1.0,"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",NaN,2,111.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro
1,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GY...",140,2583748663,2.0,"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",2.0,4,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação
2,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GA...",50,2562971980,1.0,"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",0.0,1,50.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia
3,"[POOL, BARBECUE_GRILL, GATED_COMMUNITY, GYM, G...",58,2580478200,1.0,"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",NaN,1,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro
4,"[PETS_ALLOWED, GATED_COMMUNITY, ELECTRONIC_GAT...",64,2583729583,1.0,"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",NaN,1,80.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho


In [21]:
df['rental_price'] = df['pricingInfos'].apply(get_rental_price)

In [22]:
df.drop(['amenities','address','pricingInfos'], axis=1,inplace=True)

In [23]:
df.head()

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,rental_price
0,101,2574084550,1.0,NaN,2,111.0,2,1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro,2300.0
1,140,2583748663,2.0,2.0,4,NaN,2,1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação,9500.0
2,50,2562971980,1.0,0.0,1,50.0,2,1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia,3000.0
3,58,2580478200,1.0,NaN,1,NaN,2,0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro,1900.0
4,64,2583729583,1.0,NaN,1,80.0,2,1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho,2400.0


## Análise dos dados

### Análise univariada

### Análise bivariada

### Análise de correlação